# ロジスティック回帰

## 概要

### ロジスティック回帰とは

ロジスティック回帰分析は目的変数が1となる確率を予測します。

### ロジスティック回帰の活用例

* キャンペーンの反応率
* 土砂災害発生危険基準線の確率
* 医療における症例の発生確率

## ロジスティック回帰モデル(2クラス分類)

$$ \sigma\big( a \big)  = \frac{1}{1 + exp(-a)} $$

$$ p \big( C_{1}|\phi \big) = y\big( \phi \big) = \sigma\big( w^{T}\phi \big)  $$

$$ p \big( C_{1}|\phi \big) = \frac{1}{1 + exp(- w^{T}\phi)} $$

<div style="text-align: right;">p204. 4.3.2 ロジスティック回帰</div>

## 最尤法を用いたパラメータ(w)決定

### 交差エントロピー誤差関数(cross-entropy error function)

$$ \nabla E\big(w\big) = \sum_{n=1}^{N} \big( y_{n} - t_{n} \big) \phi_{n} $$

### ニュートン-ラフソン法による w の更新

$$ w^{(new)} = w^{(old)} - \big( \Phi^{T}R\Phi \big)^{-1}\Phi^{T}\big(y - t\big) $$
$$ = \big( \Phi^{T}R\Phi \big)^{-1} \big\{ \Phi^{T}R\Phi w^{(old)} - \Phi^{T}(y - t) \big\} $$
$$ = \big( \Phi^{T}R\Phi \big)^{-1}\Phi^{T}Rz $$

重み付き最小二乗問題に対する正規方程式の集合。

#### NxN の対角行列R

$$ R_{nn} = y_{n} \big( 1 - y_{n} \big) $$

パラメータ・ベクトル w に依存しているので、wが新しくなるたびに重み付け行列Rを計算し直す必要がある。

#### N次元ベクトルN

$$ z = \Phi w^{(old)} - R^{-1}\big( y - t \big)$$

このアルゴリズムを反復再重み付け最小二乗法( *iterative reweighted least squares method* = **IRLS** )

変数空間

$$ a = w^{T}\phi $$

内の線形化された問題の解としてIRLSを解釈することができる。
そのときzのn番目の要素に相当する

$$z_{n}$$

は、ロジスティックシグモイド関数を現在の点

$$w^{(old)}$$

の周りで局所線形近似して得られる空間での目的変数であると解釈できる。

$$ a_{n} \big(w\big)\simeq a_{n} \big( w^{(old)} \big) + \frac{da_{n}}{dy_{n}}|_{w^{(old)}}\big( t_{n} - y_{n} \big) $$
$$ = \phi_{n}^{T}w^{(old)} - \frac{(y_{n} - t_{n})}{y_{n}(1 - y_{n})} = z_{n} $$

## 参考書籍

シュプリンガー パターン認識と機械学習